#Import der notwenigen Pakete

In [1]:
from pylab import *
import matplotlib.animation as animation
from matplotlib.widgets import Slider, Button, RadioButtons

In [2]:
def prepGoL (size = [9,9], bouncon = 1, incon = [[3,3],[3,4],[3,5]], rule = 'B3/S23'):
    """Die Funktion prepGoL bereitet Textdateien vor, in welche die Bedingungen zum Ausführen von GameOfLife benötigt werden.
    Es sind standrtmäßige Werte eingesetzt, welche nach belieben verändert werden können. Die Veränderung können sowohl durch Veränderung der Parameter der Funktion oder durch Bearbeitung des erstellten Dokuments vorgenommen werden.  
    kwargs:
        size = [9,9] : Systemgröße
        bouncon = 1 : Randbedingungen
        incon = [[3,3],[3,4],[3,5]] : Anfangsbedingungen
        rule = 'B3/S23' : Regelstring
        
    Hinweise: Es ist eine Form nach folgendem Muster für die Anweisungen zu wählen:
        size: [Länge, Breite]
        bouncon:
            0 - periodisch
            1 - begrenzt
            2 - unendlich (dynamisch)
        incon: [[xpos,ypos], [xpos,ypos], ...]
        rule: B3/S23
            B - Anzahlen von Nachbarzellen zur Geburt - /S - Anzahlen von Nachbarzellen zum Überleben vorhandener Zellen
    kein return Wert
    """
    a = open('GameOfLifeSize.dat','w')
    a.write('#Systemgröße als zwei natürliche Zahlen; Höhe und Breite (Höhe Breite)\n')
    for i in range(2):
        a.write('{} '.format(size[i]))
    a.write("\n")
    b = open('GameOfLifeBouncon.dat','w')
    b.write('#Randbedingungen als Zahl:\n')
    b.write('#0 - periodisch\n')
    b.write('#1 - begrenzt\n')
    b.write('#2 - unendlich\n')
    b.write('{}\n'.format(bouncon))
    c = open('GameOfLifeIncon.dat','w')
    c.write('#Anfangsbedingungen; Positionen der einzelnen besetzten Zellen untereinander: xpos ypos\n')
    for i in range(len(incon)):
        for j in range(2):
            c.write('{} '.format(incon[i][j]))
        c.write("\n")
    d = open('GameOfLifeRule.dat','w')
    d.write('#Regeln für das Leben der Zellen: z.B. "3 ENTER 2 3"\n')
    d.write('#erste Zeile: Anzahlen von Nachbarzellen zur Geburt \n#zweite Zeile: Anzahlen von Nachbarzellen zum Überleben vorhandener Zellen\n')
    d.write('#Anschließend sind die Zeilen so mit 0en aufzufüllen, dass sie gleich lang sind')
    for i in range(3):
        for j in range(max(len(rule.split("/")[1])-1, len(rule.split("/")[0])-1)):
            try:
                d.write('{} '.format(int(rule.split("/")[i-1][j+1])))
            except:
                d.write('0')
        d.write("\n")
    a.close()
    b.close()
    c.close()
    d.close()

In [14]:
prepGoL(size = [9,9], bouncon = 2, incon = [[3,3],[3,4],[3,5]], rule = 'B3/S23')

##Laufzeittest, ob eine Summe oder eine Matrixoperation schneller ist.

In [5]:
import numpy as np
k=10110
M=np.ones([k,k]).astype(int)
%time print(sum(M[:,0]))
print(M[:,0])
%time print(M[:,0].dot(np.ones(k)))

10110
Wall time: 10.5 ms
[1 1 1 ..., 1 1 1]
10110.0
Wall time: 1 ms


##Es wird also weiter mit Matrixoperationen gearbeitet

In [3]:
def GoLStep(M, livecrit, borncrit, bouncon):
    """
    Die Funktion GoLStep führt einen einzelnen Schritt des Programms GameOfLife durch.
    args:
        M : Matrix, welche die Daten des Feldes entählt (0 für tote und 1 für lebende Zellen)
        livecrit : Liste mit Kriterien, wann eine lebende Zelle am Leben bleibt. (Liste mit int Zahlen)
        borncrit : Liste mit Kriterien, wann eine neue Zelle geboren wird. (Liste mit int Zahlen)
        bouncon : Randbedingungen (int Zahl)
                    0 - periodisch
                    1 - begrenzt
                    2 - unendlich (dynamisch) 
    
    returns newM (neu, nach den vorgegebenen Regeln berechnete Matrix / neues Feld)
    """
    hohe, breite=M.shape
    if bouncon==2:
        l = M[:,0].dot(np.ones(hohe)) #links
        r = M[:,-1].dot(np.ones(hohe)) #rechts
        o = M[0].dot(np.ones(breite)) #oben
        u = M[-1].dot(np.ones(breite)) #unten
        if not l==0:
            M = np.c_[np.zeros(hohe), M]
            breite=breite+1
        if not r==0:
            M = np.c_[M, np.zeros(hohe)]
            breite=breite+1
        if not o==0:
            M = np.r_[np.zeros([1,breite]),M]
            hohe=hohe+1
        if not u==0:
            M = np.r_[M,np.zeros([1,breite])]
            hohe=hohe+1

        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
    elif bouncon==1:
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
    elif bouncon==0:
        neighbor1 = np.eye(hohe)+np.eye(hohe, k=1)+np.eye(hohe, k=-1)
        neighbor1[0,-1]=1
        neighbor1[-1,0]=1
        neighbor2 = np.eye(breite)+np.eye(breite, k=1)+np.eye(breite, k=-1)
        neighbor2[0,-1]=1
        neighbor2[-1,0]=1
    neighbors = neighbor1.dot(M).dot(neighbor2)-M
    livenei = M*neighbors #Matrix, welche die Nachbarzahlen der lebenden Zellen enthält
    deadnei = neighbors*(1-M) #Matrix, welche die Nachbarzahlen der toten Zellen enthält
    newM = np.zeros((hohe,breite))
    for i in livecrit:
        newM[livenei==i]=1
    for i in borncrit:
        newM[deadnei==i]=1
    #print(M)
    #print(breite, hohe)
    #print(newM)
    return newM, breite, hohe

In [193]:
def GoLplot(field, step, livecrit, borncrit, bouncon, interval, speed):
    """
    Die Funktion GoLplot stellt die Entwicklung des Feldes nach vorgegebener Regel dar.
    args:
        field : Ausganagsmatrix (Feld zum Schritt 0) mit den Daten der lebenden Zellen (1) und der toten Zellen (0)
        step : Funktion, welche die Matrix zum nächsten Zeitpunkt bestimmt (mit den args *[field, livecrit, borncrit, bouncon])
        livecrit : Liste mit Kriterien, wann eine lebende Zelle am Leben bleibt. (Liste mit int Zahlen)
        borncrit : Liste mit Kriterien, wann eine neue Zelle geboren wird. (Liste mit int Zahlen)
        bouncon : Randbedingungen (int Zahl)
                    0 - periodisch
                    1 - begrenzt
                    2 - unendlich (dynamisch)  
        interval : Anzahl der durchzuführenden Schritte (int Zahl)
        speed : Schnelligkeit, mit der die Animation gezeigt werden soll
                    float Zahl, welche die Verweildauer in ms angibt 
    
    kein return Wert
    """
    sfield = np.array(field)
    fields=[np.array(sfield)]
    ihohe, ibreite=sfield.shape
    hohen=[ihohe]
    breiten=[ibreite]
    ibouncon=int(bouncon)
    pause=False
    fig, ax = subplots()
    ax.set_position([0.3,0.25,0.69,0.69])
    line = ax.imshow(field, interpolation='nearest')
    
    slideri = axes([0.35, 0.1, 0.6, 0.03])
    reali = Slider(slideri, 'i', 0.0, interval, valinit=0)

    opt1 = axes([0.025, 0.85, 0.25, 0.1])
    GenOpt = Button(opt1, 'General Options')
    opt2 = axes([0.025, 0.7, 0.25, 0.1])
    ColOpt = Button(opt2, 'Colors')
    opt3 = axes([0.025, 0.55, 0.25, 0.1])
    FieldOpt = Button(opt3, 'Edit Field')
    startax = axes([0.025, 0.25, 0.1, 0.1])
    start = Button(startax, '>')
    pauseax = axes([0.15, 0.25, 0.1, 0.1])
    pauseb = Button(pauseax, '||')
    singlestepax = axes([0.15, 0.1, 0.1, 0.1])
    singlestep = Button(singlestepax, '>>')
    singlebstepax = axes([0.025, 0.1, 0.1, 0.1])
    singlebstep = Button(singlebstepax, '<<')
    
    
    #Funktionen für die verschiedenen Optionen (werden in neuem Fenster geöffnet)
    #General Options
    def genopt(val):
        fig, bounconax = subplots()
        bounconax.set_position([0.1, 0.3, 0.35, 0.4])
        bounconax.set_title("Boundary Condition")
        bounconb = RadioButtons(bounconax, ('finite', 'periodical', 'infinite'), active=0)
        
        borncritax0 = axes([0.5, 0.65, 0.1, 0.1])
        borncritax0.set_title("born-")
        borncrit0 = Button(borncritax0, '0')
        borncritax1 = axes([0.6, 0.65, 0.1, 0.1])
        borncritax1.set_title("criteria")
        borncrit1 = Button(borncritax1, '1')
        borncritax2 = axes([0.5, 0.55, 0.1, 0.1])
        borncrit2 = Button(borncritax2, '2')
        borncritax3 = axes([0.6, 0.55, 0.1, 0.1])
        borncrit3 = Button(borncritax3, '3')
        borncritax4 = axes([0.5, 0.45, 0.1, 0.1])
        borncrit4 = Button(borncritax4, '4')
        borncritax5 = axes([0.6, 0.45, 0.1, 0.1])
        borncrit5 = Button(borncritax5, '5')
        borncritax6 = axes([0.5, 0.35, 0.1, 0.1])
        borncrit6 = Button(borncritax6, '6')
        borncritax7 = axes([0.6, 0.35, 0.1, 0.1])
        borncrit7 = Button(borncritax7, '7')
        borncritax8 = axes([0.5, 0.25, 0.1, 0.1])
        borncrit8 = Button(borncritax8, '8')
        borncritax9 = axes([0.6, 0.25, 0.1, 0.1])
        borncrit9 = Button(borncritax9, '9')
        
        livecritax0 = axes([0.75, 0.65, 0.1, 0.1])
        livecritax0.set_title("live-")
        livecrit0 = Button(livecritax0, '0')
        livecritax1 = axes([0.85, 0.65, 0.1, 0.1])
        livecritax1.set_title("criteria")
        livecrit1 = Button(livecritax1, '1')
        livecritax2 = axes([0.75, 0.55, 0.1, 0.1])
        livecrit2 = Button(livecritax2, '2')
        livecritax3 = axes([0.85, 0.55, 0.1, 0.1])
        livecrit3 = Button(livecritax3, '3')
        livecritax4 = axes([0.75, 0.45, 0.1, 0.1])
        livecrit4 = Button(livecritax4, '4')
        livecritax5 = axes([0.85, 0.45, 0.1, 0.1])
        livecrit5 = Button(livecritax5, '5')
        livecritax6 = axes([0.75, 0.35, 0.1, 0.1])
        livecrit6 = Button(livecritax6, '6')
        livecritax7 = axes([0.85, 0.35, 0.1, 0.1])
        livecrit7 = Button(livecritax7, '7')
        livecritax8 = axes([0.75, 0.25, 0.1, 0.1])
        livecrit8 = Button(livecritax8, '8')
        livecritax9 = axes([0.85, 0.25, 0.1, 0.1])
        livecrit9 = Button(livecritax9, '9')
        show()
    
        def bounconfunc(bcon):
            '''
            Die Funktion bounconfunc ändert die Animation entsprechend der neuen Randbedingung.
            Sie wird von vorne gestartet.
            args:
                bcon : gewählte Randbedingung
            '''
            if bcon in ['finite',1]:
                pass
            if bcon in ['infinite',2]:
                pass
            if bcon in ['periodical',0]:
                pass
    
    def colors(val):
        fig, sliderr = subplots()
        sliderr.set_position([0.35, 0.35, 0.6, 0.03])
        sliderg = axes([0.35, 0.498, 0.6, 0.03])
        sliderb = axes([0.35, 0.647, 0.6, 0.03])
        colors = axes([0.025,0.2,0.25,0.6])

        r = Slider(sliderr, 'red', 0.0, 1.0, valinit=0)
        g = Slider(sliderg, 'green', 0.0, 1.0, valinit=0)
        b = Slider(sliderb, 'blue', 0.0, 1.0, valinit=0)
        colorbuttons = RadioButtons(colors, ('black', 'green', 'red', 'blue', 'yellow'))
        show()
        
        def colorslide(val):
            '''
            Die Funktion colorslide verändert das Farbschema gemäß den auf den Sliderne eingestellten Werten.
            args: 
                val : Nötig zum Einbau in anderen Funktionen
            kein return Wert
            '''
            red=r.val
            green=g.val
            blue=b.val
            col = {'red': [(0.0,  0.0, 0.0),
                           (1.0,  red, red)],

                 'green': [(0.0,  0.0, 0.0),
                           (1.0,  green, green)],

                 'blue':  [(0.0,  0.0, 0.0),
                           (1.0,  blue, blue)],

                 'alpha': [(0.0,  0.0, 0.0),
                           (1.0,  1.0, 1.0)] }
            cm = mpl.colors.LinearSegmentedColormap("cm", col)
            register_cmap(cmap=cm)
            set_cmap(cm)

        def colorbut(col='black'):
            '''
            Die Funktion colorbut stellt das Farbschema entsprechend der gewählten Farbe ein.
            kwargs:
                col='black' : gewählte Farbe
            kein return Wert
            '''
            if col=="red":
                r.set_val(1)
                g.set_val(0)
                b.set_val(0)
            elif col=="green":
                r.set_val(0)
                g.set_val(1)
                b.set_val(0)
            elif col=="blue":
                r.set_val(0)
                g.set_val(0)
                b.set_val(1)
            elif col=="black":
                r.set_val(0)
                g.set_val(0)
                b.set_val(0)
            elif col=="yellow":
                r.set_val(1)
                g.set_val(1)
                b.set_val(0)
            else: 
                colorbut("black")
            colorslide(0)
        
        r.on_changed(colorslide)
        g.on_changed(colorslide)
        b.on_changed(colorslide)
        colorbuttons.on_clicked(colorbut)

    #Animierter Plot des Feldes
    GenOpt.on_clicked(genopt)
    ColOpt.on_clicked(colors)
    
    def animate(i):
        '''
        Die Funktion animate wird verwendet, um das Feld animiert auszugeben. Sie gibt den Plot (genauer die Imschow) in abhängigkeit des Schrittes mit dem entsprechenden Feld aus.
        args:
            i : Nummer des Schrittes
        returns line
        '''
        if pause==False:
            if reali.val<interval:
                reali.set_val(int(reali.val)+1)
            else:
                reali.set_val(0)
        if reali.val>=len(fields):
            reali.set_val(len(fields))
        i=int(reali.val)
        if len(fields)>=i+1:
            newfield=fields[i]
            hohe=hohen[i]
            breite=breiten[i]
        else:            
            field=fields[i-1]         
            newfield, breite, hohe = step(field, livecrit, borncrit, bouncon)
            fields.append(np.array(newfield))
            hohen.append(int(hohe))
            breiten.append(int(breite))
        #print()
        #print(i)
        #print(len(fields))
        ax.set_xlim([-0.5, breite-0.5])
        ax.set_ylim([-0.5, hohe-0.5])
        ax.clear()
        line = ax.imshow(newfield, interpolation='nearest')
        ax.set_title("$step$ = {}".format(i))
        return line
    
    ani = animation.FuncAnimation(fig, animate, np.arange(0, interval+1),
                              interval=speed, repeat=True)
    
    show()
    

Die neue Idee ist, die Optionen in einem neuen Feld zu öffnen. Ferner wird überlegt, dass neben dem Prüfen von i auch noch nach pause geprüft wird, was die Animation stoppen lässt. Die Variable single_step setzt pause ein i später auf den entsprechenden Wert.

In [194]:
GameOfLife(steps=100, speed=0)

In [83]:
fig, ax = subplots()
fig.set_size_inches?

In [29]:
resetsa = plt.axes([0.025, 0.6, 0.25, 0.04])
resetsb = Button(resetsa, 'Reset All')
resetsb.on_clicked(resetall)

def Option(option):
    '''
    Die Funktion Option zeigt in Abhängigkeit der gewählten Funktion die zugehörigen Funktionselemente an.
    args:
        option : gewählte Option
                'General Options', 'Manual Running', 'Edit field', 'Colors'
    kein return Wert
    '''
    reseta.clear()
    if option=="General Options":
        genopts.set_visible(1)
        gena.on_clicked(generalOptions)
    else: 
        genopts.set_visible(0)
        bounconax.set_visible(0)
    
    if option=="Speed":
        pass
    if option=="Edit field":
        pass


#General Options
def generalOptions(option='Boundary Conditions'):
    '''
    Die Funktion generalOptions zeigt in Abhängigkeit der gewählten Option die zugehörigen Slider an.
    kwargs: 
        option='Boundary Conditions' : ausgewählte Option
    kein return Wert
    '''
    
def bounconfunc(bcon):
    '''
    Die Funktion bounconfunc ändert die Animation entsprechend der neuen Randbedingung.
    Sie wird von vorne gestartet.
    args:
        bcon : gewählte Randbedingung
    '''
    if bcon in ['finite',1]:
        pass
    if bcon in ['infinite',2]:
        pass
    if bcon in ['periodical',0]:
        pass

#Farben
red=0
green=0
blue=0

#Reset All fehlt
def resetall(val):
    '''
    Die Funktion resetall setzt alle Werte auf den Ursprungswert zurück.
    args:
        val : Benötigt zum Einbau in anderen Funktionen
    kein return Wert
    '''
    colorbut('black')
    bounconfunc(ibouncon)

#Optionen anzeigen
opt = axes([0.025, 0.65, 0.25, 0.25])
Options = RadioButtons(opt, ('General Options', 'Manual Running', 'Edit field', 'Colors'), active=0)
Options.on_clicked(Option)
resetsa = plt.axes([0.025, 0.6, 0.25, 0.04])
resetsb = Button(resetsa, 'Reset All')
resetsb.on_clicked(resetall)

#Optionselemente
reseta = axes([0.025,0.025,0.2,0.1])
resetb = Button(reseta, 'Reset')

#General Options (Randbedingungen, Regelstring, Size)
genopts = axes([0.025,0.2,0.25,0.38])
gena = RadioButtons(genopts, ('Boundary Conditions','Rule','Size'), active=0)
bounconax = axes([0.35, 0.05, 0.3, 0.13])
bounconb = RadioButtons(bounconax, ('finite', 'infinite', 'periodical'))

#Speed (Schieberegler für Geschwindigkeit)


#Edit field (Stop, Bearbeiten, Übernehmen, u.U. Knopf mit neuem Fenster mit Vorschlägen, wenn angecklickt, dann übernehmbar in Feld, bei weiterem Klick bestätigung)


#Running (Start, Stop, Schieberegler für Runde, single step)


#Anfangsbedingung schaffen
Option("General Options")
colorbut("black")


IndentationError: unexpected indent (<ipython-input-29-ee654b5bba28>, line 2)

##Problem:
Die Buttons, etc. arbeiten im Hintergrund weiter, wenn sie mit einer else Verzweigung disconnected werden, dann arbeiten sie gar nict mehr...

In [149]:
test=RadioButtons(genopts, ('hallo', 'tschuss'))
test=Button(genopts, 'hallo')

In [141]:
genopts = axes([0.025,0.2,0.25,0.38])
colors = axes([0.025,0.2,0.25,0.38])
genopts is colors

True

##Funktion zum Import der Benutzereingabe

In [5]:
def GoLdataimport():
    """
    Die Funktion GoLdataimport importiert die Bedingungen aus den mit prepGoL erstellten Textdateien. Die Form ist wie bei prepGoL beschrieben zu halten.
    Sie benötigt keine Argumente.
    returns (np.arrays)
        size : Größe des Feldes
        bouncon : Randbedingung
        incon : Anfangsbedingung
        rule : Regel (erste Zeile: bornrule, zweite Zeile: liverule)
    """
    size = np.loadtxt("GameOfLifeSize.dat", dtype=int)
    bouncon = np.loadtxt("GameOfLifeBouncon.dat", dtype=int)
    incon = np.loadtxt("GameOfLifeIncon.dat", dtype=int)
    rule = np.loadtxt("GameOfLifeRule.dat", dtype=int)
    return size, bouncon, incon, rule

In [7]:
size, bouncon, incon, rule=GoLdataimport()

##Funktion, welche den Datenimport und den Plot vornimmt

In [6]:
def GameOfLife(steps=10, speed=1):
    """
    Die Funktion GameOfLife erzeugt eine graphisch animierte Ausgabe der Entwicklung der Anfangsmatrix nach dem Game of Life.
    Die Bedingungen sind durch Textdateien vorzugeben (siehe auch prepGoL).
    kwargs:
        steps=10 : Anzahl der durchzufürenden Schritte
        speed=1 : Geschwindigkeit, mit der die Aimation erfolgen soll
                    100*10**speed ist die Zeit in ms pro Schritt (empfohlen: Wert zwischen 0 (langsam) und 1 (schnell))
    """
    size, bouncon, incon, rule = GoLdataimport()
    liverule = [rule[1][i] for i in range(len(rule[1])) if rule[1][i]!=0]
    bornrule = [rule[0][i] for i in range(len(rule[0])) if rule[0][i]!=0]
        
    
    field =  np.zeros(size)
    for i in range(int(incon.size/incon[0].size)):
        field[incon[i][1], incon[i][0]]=1
    
    speed=100*10**(1-speed)
    
    GoLplot(field, GoLStep, liverule, bornrule, bouncon, steps, speed)
    return

In [12]:
GameOfLife(steps=100, speed=0)

Problem: Er rechnet immer vom Ende weiter, die Anfangsbedungung ist also noch zu speichern. - gelöst

Ferner funktionrt die Unendlichkeitsbedingung noch nicht.
    - Erweiterung der Matrix wird nicht angezeigt und nicht übernommen - gelöst

In [98]:
np.c_[np.ones([2,3]), np.zeros([2,3])]

array([[ 1.,  1.,  1.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  0.,  0.,  0.]])

In [13]:
x=np.ones([3,3])
print(x)
y=np.array(x)

y.resize([33,33])
x.data=y
print(y)
print(x)
print(np.array(x.data))

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.  1.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]


In [144]:
a=np.ones([2,2])
a.data=np.zeros([3,3])
a[1,1]=2
print(a)
a.base

[[ 0.  0.]
 [ 0.  2.]]


array([[ 0.,  0.,  0.],
       [ 2.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [53]:
x=[1,2,3,4,5]
x.append(6)
x[5]

6

Das Grundprinzip funktioniert nun. Es fehlen einige Aspekte wie
- Randbedingungen in GoLStep
- Funktionsbeschreibungen
- evtl. Sonderfunktionen wie 
    - Farbwahl
    - Starten, Stoppen per Maus
    - Bearbeiten der Matrix per Maus
    - ...